<a href="https://colab.research.google.com/github/hemander1433/DATA-SCIENCE-ASSIGNEMNT/blob/main/sai_hemander_kambhampati_Lookalike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LOOKALIKE MODEL TASK 2**


In [7]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

In [13]:
customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv('Products.csv')
transactions_df = pd.read_csv('Transactions.csv')
print(customers_df.head())
print(products_df.head())
print(transactions_df.head())

  CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15
  ProductID              ProductName     Category   Price
0      P001     ActiveWear Biography        Books  169.30
1      P002    ActiveWear Smartwatch  Electronics  346.30
2      P003  ComfortLiving Biography        Books   44.12
3      P004            BookWorld Rug   Home Decor   95.69
4      P005          TechPro T-Shirt     Clothing  429.31
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3       

In [15]:
# Merge transaction data with product and customer details
merged_df = transactions_df.merge(customers_df, on='CustomerID').merge(products_df, on='ProductID')
print(merged_df.head())

  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x     CustomerName         Region  SignupDate  \
0      300.68   300.68   Andrea Jenkins         Europe  2022-12-03   
1      300.68   300.68  Brittany Harvey           Asia  2024-09-04   
2      300.68   300.68  Kathryn Stevens         Europe  2024-04-04   
3      601.36   300.68  Travis Campbell  South America  2024-04-11   
4      902.04   300.68    Timothy Perez         Europe  2022-03-15   

                       ProductName     Category  Price_y  
0  ComfortLiving Bluetooth Speaker  Electronics   300.68  
1  ComfortLiving Bluetooth Speaker

In [16]:
# Create a customer-product matrix (aggregated purchase behavior)
customer_product_matrix = merged_df.pivot_table(
    index='CustomerID',
    columns='ProductID',
    values='TotalValue',
    aggfunc='sum',
    fill_value=0
)

In [17]:
customers_df['RegionEncoded'] = customers_df['Region'].astype('category').cat.codes

# Merge region data into the matrix
customer_profile_matrix = customer_product_matrix.merge(
    customers_df[['CustomerID', 'RegionEncoded']],
    left_index=True,
    right_on='CustomerID'
).set_index('CustomerID')

In [19]:
# Normalize features for caluculation purpose...

scaler = MinMaxScaler()
normalized_matrix = scaler.fit_transform(customer_profile_matrix)

In [20]:
# Calculate cosine similarity
similarity_matrix = cosine_similarity(normalized_matrix)

# Create a similarity DataFrame
similarity_df = pd.DataFrame(similarity_matrix, index=customer_profile_matrix.index, columns=customer_profile_matrix.index)


In [21]:
def recommend_similar_customers(customer_id, top_n=3):
    if customer_id not in similarity_df.index:
        return f"Customer {customer_id} not found."
    # Sort by similarity score and exclude the customer itself
    similar_customers = similarity_df[customer_id].sort_values(ascending=False).iloc[1:top_n+1]
    recommendations = similar_customers.index.tolist()
    scores = similar_customers.values
    return pd.DataFrame({'RecommendedCustomerID': recommendations, 'SimilarityScore': scores})


In [25]:
 ## for an example , let us take a random number , here i have choosen number 5.
 ## lets see which customer jd it will print and what wil be the similarity socre...

input_customer_id = 'C0005'
recommendations = recommend_similar_customers(input_customer_id)

# Output the recommendations
print(f"Recommendations for Customer {input_customer_id}:\n", recommendations)

Recommendations for Customer C0005:
   RecommendedCustomerID  SimilarityScore
0                 C0096         0.571662
1                 C0055         0.479191
2                 C0064         0.328245


In [29]:
# Function to generate lookalikes for multiple customers
def generate_lookalike_map(customer_ids, top_n=3):
    lookalike_map = {}
    for customer_id in customer_ids:
        recommendations = recommend_similar_customers(customer_id, top_n)
        lookalike_map[customer_id] = recommendations.to_dict(orient='records')  # Convert to list of dictionaries
    return lookalike_map

# to Generate lookalikes for the first 20 customers (CustomerID: C0001 - C0020)

first_20_customers = customers_df['CustomerID'][:20]
lookalike_map = generate_lookalike_map(first_20_customers)


## here is the first lookalike mdoel for the first 20 customers in the custmers list....

lookalike_df = pd.DataFrame(lookalike_data)


print(lookalike_df)




   cust_id                                         lookalikes
0    C0001  [{'cust_id': 'C0083', 'score': 0.6339024810972...
1    C0002  [{'cust_id': 'C0030', 'score': 0.3443911869509...
2    C0003  [{'cust_id': 'C0181', 'score': 0.6410058864642...
3    C0004  [{'cust_id': 'C0076', 'score': 0.4927416817572...
4    C0005  [{'cust_id': 'C0096', 'score': 0.5716619504750...
5    C0006  [{'cust_id': 'C0058', 'score': 0.6635145607498...
6    C0007  [{'cust_id': 'C0079', 'score': 0.4571635906556...
7    C0008  [{'cust_id': 'C0144', 'score': 0.5336966823503...
8    C0009  [{'cust_id': 'C0112', 'score': 0.7238626917533...
9    C0010  [{'cust_id': 'C0077', 'score': 0.5017480088416...
10   C0011  [{'cust_id': 'C0120', 'score': 0.5969620057957...
11   C0012  [{'cust_id': 'C0158', 'score': 0.5574903060063...
12   C0013  [{'cust_id': 'C0169', 'score': 0.6352267941995...
13   C0014  [{'cust_id': 'C0128', 'score': 0.7725393039369...
14   C0015  [{'cust_id': 'C0036', 'score': 0.7218666801535...
15   C00